# Bibliotecas usadas

In [1]:
import pandas as pd
import glob
import re
from datetime import datetime, timedelta

ModuleNotFoundError: No module named 'pandas'

# Importar bases

In [215]:
# localizar todos os arquivos csv dos experimentos
padrao_busca = "/Users/michi/Desktop/01. Outdoor-Indoor/01. LabEEE Experiments/**/*.csv"
todos_arquivos = glob.glob(padrao_busca, recursive=True)

# filtra os arquivos de interesse
ids = ["IS", "LG", "LM", "MF", "TM", "ZM", "AW", "JF", "JP", "MN", "NC", "RM"]
nao = ["01. Pilot LabEEE 20231114", "02. Pilot LabEEE 20231122"]
arquivos_csv = [arquivo for arquivo in todos_arquivos if any(sigla in arquivo for sigla in ids) and not any(nome in arquivo for nome in nao)]

print(len(arquivos_csv))

520


In [216]:
relacao_tabela_informacao = {"A" :	["4B00000069A8E941","FE00000069AF1041","B600000069B23241","7D00000069AB4641","0F00000069AAAF41","C400000069ADCC41"],
                            "D":	["CB00000069AD3041","9400000069AA4941","2700000069AD8541","C100000069B26141","CA00000069B19041","4A00000069AC0841"],
                            "F":	["C400000069AA8341","C400000069B3DB41","8100000069AE1641","2700000069AACA41","6700000069AEFC41"	"1C00000069B3F941"], 
                            "H":	["B200000069AE3141","EC00000069A9CD41","3300000069AC2241","3600000069AAD741","E800000069AAC041","AD00000069AF4C41"],
                            "J":	["FA00000069AD0441","3400000069B1AE41","C700000069B34D41","4B00000069B0A841","0600000069B22741","9100000069AAEA41","6700000069AB8141"],
                            "K":	["FB00000069B28F41","7B00000069AA6A41","E500000069AA2F41","1A00000069A9BF41","FE00000069B10741","E100000069AD4641"],
                            "M":	["1B00000069B06241","8C00000069ADD241","F000000069AF6941","AA00000069AA8141","1600000069ABE741","A000000069AEBA41"],
                            "O":	["7600000069AA8541","F200000069B16D41","3000000069A9C941","BF00000069AFC741","A300000069B12241","D900000069ACA241"],
                            "Q":	["8600000069B0B141","6000000069AB3141","DB00000069A9CC41","EF00000069B23141","6900000069AE8541","4300000069B3CF41"],
                            "T":	["7600000069AB9C41","4500000069B1C841","6600000069B25C41","A200000069B0BE41","D700000069B3D541","9800000069B26241","9E00000069AE7241"],
                            "T_RH" : ["6E0000006CB57E41",	"590000006CB46641",	"4B0000006CBA7341",	"D30000006CBD4C41",	"370000006341C241",	"9C000000633F9F41"]}


nomes = ["Zac", "Joao", "Igor", "Luis", "Milena", "Ana", "Marcela", "Nathalia", "Thalita", "Liege", "Rayner", "João"]
sports = ["WALKING","CYCLING","CROSS-COUNTRY_SKIING","SKATING","RIDING","ROWING"]

In [217]:
def verifica_qual_tipo_de_csv(titulo):
    for chave, valores in relacao_tabela_informacao.items():
        if titulo in valores:
            return chave

def convertepfloat(row):
    n1 = str(row.iloc[2]).replace('.', '')
    n2 = str(row.iloc[3]).replace('.', '')
    return float(n1 + '.' + n2)

def datamento(base_de_dados_crua, base_de_dados_nova):
    dia = base_de_dados_crua.iloc[0,2]
    dia = dia.replace('-','/')
    hora_inicio = base_de_dados_crua.iloc[0,3]
    data_inicio = dia + " " + hora_inicio
    data_inicio = datetime.strptime(data_inicio, '%d/%m/%Y %H:%M:%S')
    data_fim = dia + ' ' + '20:00:00'
    data_fim = datetime.strptime(data_fim, '%d/%m/%Y %H:%M:%S')
    intervalos_tempo = pd.date_range(start=data_inicio, end=data_fim, freq='s')
    base_de_dados_nova['Date'] = intervalos_tempo[:len(base_de_dados_nova)]

def limpeza_simples(id, arquivo, coluna):
    colunas = ["Date", "uni", "u", "d"]
    dados = pd.read_csv(arquivo, skiprows=20,names=colunas).fillna('0')
    dados['id'] = id
    dados[f'{coluna}'] = dados.apply(lambda row: convertepfloat(row), axis=1)
    dados = dados.iloc[:, [0,-2,-1]]
    dados['Date'] = pd.to_datetime(dados['Date'], format='%d/%m/%y %H:%M:%S')
    return dados

def calcular_TPond(row):
    if all(row[['A (°C)', 'D (°C)', 'F (°C)', 'H (°C)', 'J (°C)', 'K (°C)', 'M (°C)', 'O (°C)', 'Q (°C)', 'T (°C)']].notnull()):
        return 0.07 * row['T (°C)'] + 0.13 * row['Q (°C)'] + 0.19 * row['O (°C)'] + 0.12 * row['M (°C)'] + 0.12 * row['K (°C)'] + 0.12 * row['J (°C)'] + 0.05 * row['H (°C)'] + 0.06 * row['F (°C)'] + 0.08 * row['D (°C)'] + 0.06 * row['A (°C)']
    else:
        return None

In [218]:
col_names = ['id', 'Date', 'TPond (°C)', 'A (°C)', 'D (°C)', 'F (°C)', 'H (°C)', 'J (°C)', 'K (°C)', 'M (°C)', 'O (°C)', 'Q (°C)',	'T (°C)', 'wHR (bpm)', 'cHR (bpm)', 'wU (bpm)', 'wT (°C)']
BaseDeDados = pd.DataFrame(columns=col_names)
print(BaseDeDados)

Empty DataFrame
Columns: [id, Date, TPond (°C), A (°C), D (°C), F (°C), H (°C), J (°C), K (°C), M (°C), O (°C), Q (°C), T (°C), wHR (bpm), cHR (bpm), wU (bpm), wT (°C)]
Index: []


In [ ]:
# substituir if/elif/else por match case

for arquivo in arquivos_csv:
    ultimo_backslash = arquivo.rfind('\\')
    ultimounderscore = arquivo.rfind('_')
    titulo = arquivo[ultimo_backslash+1:ultimounderscore]
    penultimo_backslash = arquivo.rfind('\\', 0, ultimo_backslash)
    id = arquivo[penultimo_backslash+1:penultimo_backslash+3]
    if "Test+Room" in titulo or any(name in titulo for name in nomes):
        heart = pd.read_csv(arquivo).fillna('0')
        if(heart.iloc[0,1] == "OTHER_OUTDOOR"):
            Dia = heart.iloc[0,2]
            HInicio = heart.iloc[0,3]
            DataInicio = Dia + " " + HInicio
            pulso = pd.read_csv(arquivo, usecols=["Time","HR (bpm)"], skiprows=2)
            pulso['id'] = id
            pulso = pulso.rename(columns={'Time': 'Date'})
            pulso = pulso.rename(columns={'HR (bpm)': 'HR (bpm) W'})
            # arrumar a data
            BaseDeDados = pd.concat([BaseDeDados, pulso], ignore_index=True)
        elif(heart.iloc[0,1] in sports):
            Dia = heart.iloc[0,2]
            HInicio = heart.iloc[0,3]
            DataInicio = Dia + " " + HInicio
            peito = pd.read_csv(arquivo, usecols=["Time","HR (bpm)"], skiprows=2)
            peito['id'] = id
            peito = peito.rename(columns={'Time': 'Date'})
            peito = peito.rename(columns={'HR (bpm)': 'HR (bpm) C'})
            # arrumar a data
            BaseDeDados = pd.concat([BaseDeDados, peito], ignore_index=True)
        elif(heart.iloc[0,1] == "False" or "True"):
            continue
        else:
            print(heart.iloc[0,1])    
    elif("_U" in arquivo) or ("_T" in arquivo) or (any(titulo in valores for valores in relacao_tabela_informacao.values())):
        colunas = ["Date", "uni", "u", "d"]
        if("_T" in arquivo):
            # somar coluna de unidade de medica e valor
            temp = pd.read_csv(arquivo, skiprows=20,names=colunas).fillna('0')
            temp['id'] = id
            temp['Temp'] = temp.apply(lambda row: convertepfloat(row), axis=1)
            temp = temp.iloc[:, [0,1,-2,-1]]
            temp['Date'] = pd.to_datetime(temp['Date'], format='%d/%m/%y %H:%M:%S')
            BaseDeDados = pd.concat([BaseDeDados, temp],ignore_index=True)
        elif("_U" in arquivo):
            # somar coluna de unidade de medica e valor
            umid = pd.read_csv(arquivo, skiprows=20,names=colunas).fillna('0')
            umid['id'] = id
            umid['Umid'] = umid.apply(lambda row: convertepfloat(row), axis=1)
            umid = umid.iloc[:, [0,1,-2,-1]]
            umid['Date'] = pd.to_datetime(umid['Date'], format='%d/%m/%y %H:%M:%S')
            BaseDeDados = pd.concat([BaseDeDados, umid],ignore_index=True)
        else:
            # somar coluna de unidade de medica e valor
            # identificar qual sensor de pele é o arquivo
            coluna = verifica_qual_tipo_de_csv(titulo)
            temp = pd.read_csv(arquivo, skiprows=20,names=colunas).fillna('0')
            temp[f'{coluna}'] = temp.apply(lambda row: convertepfloat(row), axis=1)
            temp['id'] = id
            temp = temp.iloc[:, [0,1,-2,-1]]
            temp['Date'] = pd.to_datetime(temp['Date'], format='%d/%m/%y %H:%M:%S')
            BaseDeDados = pd.concat([BaseDeDados, temp],ignore_index=True)
    else:
        print(titulo)
    #print("match")

In [ ]:
BaseDeDados['TPond (°C)'] = BaseDeDados.apply(calcular_TPond, axis=1)

In [223]:
BaseDeDados.to_csv('arquivo.csv', index=False)